In [1]:
import cv2

In [2]:

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from keras.layers import Input, Dense, Reshape, Flatten, Dropout
# from keras.layers import BatchNormalization, Activation, ZeroPadding2D
# from keras.layers.advanced_activations import LeakyReLU
# from keras.layers.convolutional import UpSampling2D, Conv2D
# from keras.models import Sequential, Model
# #from keras.optimizers import Adam
# from keras.utils import np_utils

# from keras.backend import tensorflow_backend

import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
# import rarfile as rar
from pathlib import Path

np.random.seed(0)
np.random.RandomState(0)
tf.random.set_seed(0)

config = tf.compat.v1.ConfigProto(gpu_options=tf.compat.v1.GPUOptions(allow_growth=True))
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

root_dir = str(Path('img').resolve())
class_names = os.listdir(root_dir)
print(class_names)


['Black']


In [6]:
# import keras
# print(keras.__version__) 

In [73]:
class DCGAN():
    
    def __init__(self):

        self.class_names = os.listdir(root_dir)

        self.shape = (8, 8, 1) #(8, 8, 3)
        self.z_dim = 100

        optimizer = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        ori_noise, self.generator = self.build_generator()
        # self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        #tf.compat.v1.disable_eager_execution()
        
        z = tf.keras.Input(shape=(self.z_dim,))
        img = self.generator(z)
        #print(tf.shape(img))
        
        
        init = tf.compat.v1.global_variables_initializer()
        
#         with tf.compat.v1.Session() as sess:
#             sess.run(init)
        sess = tf.compat.v1.Session()
        sess.run(init)

        elems = img
        
        
        print("imgs shape (before mapping): " + str(elems.shape))
        op = tf.map_fn(self.wavelet, elems)
        result = sess.run(op)

        for elem in elems:
            print('hoge', type(elem))

        
#         print(type(img))
#         print(tf.shape(img))
        img = self.wavelet(img)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    # must under eagar mode
    def tensor_to_array(self, tensor1):
        return np.array(tensor1)


    def build_generator(self):
        noise_shape = (self.z_dim,)

        model = tf.keras.Sequential()

        model.add(layers.Dense(128 * 8 * 8, activation="relu", input_shape=noise_shape))
        model.add(layers.Reshape((8, 8, 128)))
        model.add(layers.BatchNormalization(momentum=0.8))
        #model.add(UpSampling2D())
        model.add(layers.Conv2D(128, kernel_size=3, padding="same"))
        model.add(layers.Activation("relu"))
        model.add(layers.BatchNormalization(momentum=0.8))
        #model.add(UpSampling2D())
        model.add(layers.Conv2D(64, kernel_size=3, padding="same"))
        model.add(layers.Activation("relu"))
        model.add(layers.BatchNormalization(momentum=0.8))
        model.add(layers.Conv2D(1, kernel_size=3, padding="same"))
        model.add(layers.Activation("tanh"))

        #model.summary()

        noise = keras.Input(shape=noise_shape)
        img = model(noise)
        print(img.shape)

        return noise, keras.Model(noise, img)

    def build_discriminator(self):
        img_shape = self.shape

        #4 機械学習（人工知能）モデルの作成
        #2021/6/2「Conv2D」を使ってディープラーニング技術でおなじみの「畳み込みニューラルネットワーク」（CNN）を実装
        model = tf.keras.Sequential()
        
        model = keras.Sequential()
        model.add(layers.Conv2D(48, (2, 2), padding='same', input_shape=(8, 8, 1), activation='relu'))
        model.add(layers.Conv2D(128, (2, 2), padding='same', activation='relu'))
        model.add(layers.Dropout(0.25))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation='sigmoid'))

        model.summary()
            
        img = keras.Input(shape=img_shape)
        validity = model(img)
        
        #model.summary()

        return keras.Model(img, validity)
    
    def build_discriminator2(self):
        
        img_shape = self.shape
        
        model = Sequential()
        
        model.add(Conv2D(48, (1, 1), padding='same', input_shape=(4, 4, 3), activation='relu'))
        model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
        
        img = Input(shape=img_shape)
        
#         w_image = np.zeros((4, 4, 3))
#         for num in range(0, 4):
#             for ber in range(0, 4):
#                 w_image[num][ber][0] = (img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
#                 w_image[num][ber][1] = (img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
#                 w_image[num][ber][2] = (img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4
        #w_images.append(w_image)
        validity = model(img)
        

        return Model(img, validity)

    def build_combined(self):
        self.discriminator.trainable = False
        model = Sequential([self.generator, self.discriminator])

        return model

    def train(self, iterations, batch_size=128, save_interval=50, model_interval=1000, check_noise=None, r=5, c=5):

        X_train, labels = self.load_imgs()

        half_batch = int(batch_size / 2)

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        for iteration in range(iterations):

            # ------------------
            # Training Discriminator
            # -----------------
            idx = np.random.randint(0, X_train.shape[0], half_batch)

            imgs = X_train[idx]

            noise = np.random.uniform(-1, 1, (half_batch, self.z_dim))

            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # -----------------
            # Training Generator
            # -----------------

            noise = np.random.uniform(-1, 1, (batch_size, self.z_dim))

            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100 * d_loss[1], g_loss))

            model_dir = Path('ganmodels')
            model_dir.mkdir(exist_ok=True)
            if iteration % save_interval == 0:
                self.save_imgs(iteration, check_noise, r, c)
                start = np.expand_dims(check_noise[0], axis=0)
                end = np.expand_dims(check_noise[1], axis=0)
                resultImage = self.visualizeInterpolation(start=start, end=end)
                cv2.imwrite("images/latent/" + "latent_{}.png".format(iteration), resultImage)
                if iteration % model_interval == 0:
                    self.generator.save(str(model_dir)+"/dcgan-{}-iter.h5".format(iteration))

    def save_imgs(self, iteration, check_noise, r, c):
        noise = check_noise
        gen_imgs = self.generator.predict(noise)

        # 0-1 rescale
        gen_imgs = 0.5 * gen_imgs + 0.5
        #print(type(gen_imgs)
        gen_imgs = np.squeeze(gen_imgs)
        print(gen_imgs.shape)
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :])#, :])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig('images/gen_imgs/watermark_%d.png' % iteration)

        plt.close()
        
    def get_class_one_hot(self, class_str):
        label_encoded = self.class_names.index(class_str)

        label_hot = np_utils.to_categorical(label_encoded, len(self.class_names))
        label_hot = label_hot

        return label_hot

    def load_imgs(self, ret_wavelet=False):

#         img_paths = []
#         labels = []
#         images = []
#         for cl_name in self.class_names:
#             img_names = os.listdir(os.path.join(root_dir, cl_name))
#             for img_name in img_names:
#                 img_paths.append(os.path.abspath(os.path.join(root_dir, cl_name, img_name)))
#                 hot_cl_name = self.get_class_one_hot(cl_name)
#                 labels.append(hot_cl_name)

#         for img_path in img_paths:
#             img = cv2.imread(img_path)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#             images.append(img)

#         images = np.array(images)

#        return (np.array(images), np.array(labels))
        img_paths = []
        labels = []
        images = []
        w_images = []

        for cl_name in class_names:
            print(cl_name)
            img_dir_names = os.listdir(os.path.join(root_dir, cl_name))

            for img_dir_name in img_dir_names:
                img_names = os.listdir(os.path.join(root_dir, cl_name, img_dir_name))

                for img_name in img_names:
                    img_paths.append(os.path.abspath(os.path.join(root_dir, cl_name, img_dir_name, img_name)))
                    hot_cl_name = self.get_class_one_hot(cl_name)
                    labels.append(hot_cl_name)

        for img_path in img_paths:
            img = cv2.imread(img_path)

            # (8, 8, 3) -> (8, 8, 1)へ次元を削減 すべて同じ値なので
            img = np.delete(img, [1, 2], 2)
            images.append(img)
            w_image = np.zeros((4, 4, 3))
            MM_image = np.zeros((4,4,2))
            for num in range(0, 4):
                for ber in range(0, 4):
                    w_image[num][ber][0] = (img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
                    w_image[num][ber][1] = (img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4
                    w_image[num][ber][2] = (img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4
                    MM_image[num][ber][0] = max([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
                    MM_image[num][ber][1] = min([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
            #w_images.append(MM_image)
            w_images.append(w_image)

        images = np.array(images)

        if ret_wavelet:
            return np.array(w_images), np.array(labels)
        else:
            return np.array(images), np.array(labels)

   

    def visualizeInterpolation(self, start, end, save=True, nbSteps=10):
        print("Generating interpolations...")

        steps = nbSteps
        latentStart = start
        latentEnd = end

        startImg = self.generator.predict(latentStart)
        endImg = self.generator.predict(latentEnd)

        vectors = []

        alphaValues = np.linspace(0, 1, steps)
        for alpha in alphaValues:
            vector = latentStart * (1 - alpha) + latentEnd * alpha
            vectors.append(vector)

        vectors = np.array(vectors)

        resultLatent = None
        resultImage = None

        for i, vec in enumerate(vectors):
            gen_img = np.squeeze(self.generator.predict(vec), axis=0)
            gen_img = (0.5 * gen_img + 0.5) * 255
            interpolatedImage = gen_img #cv2.cvtColor(gen_img, cv2.COLOR_RGB2BGR)
            interpolatedImage = interpolatedImage.astype(np.uint8)
            resultImage = interpolatedImage if resultImage is None else np.hstack([resultImage, interpolatedImage])

        return resultImage
    
    def wavelet(self, img):
        #images.append(img)

        print(img)
        w_images = []
        print("img shape (after mapping):", img.shape)
        
        
#         w_image = tf.Variable(tf.zeros((4,4,3)))
        #w_image = tf.zeros((4,4,3))
        wave = tf.Variable(lambda: tf.zeros([4,4,3]))
#         wave = tf.Variable(lambda : tf.random.truncated_normal([10, 40]))


#         op = tf.assign(w_image, wavelet_test(w_image, img))
#         with tf.Session() as sess:
#             sess.run(tf.global_variables_initializer())
#             ret = sess.run(op)
#             print(type(ret)) # <class 'numpy.ndarray'>
#             print(ret)
#         w_image = tf.zeros(shape=[4, 4, 3])
  
#         #MM_image = np.zeros((4,4,2))
#         for num in range(0, 4):
#             for ber in range(0, 4):
#                 print((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]))
#                 w_image[num][ber][0].assign((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#                 w_image[num][ber][1].assign((img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#                 w_image[num][ber][2].assign((img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4)
                #MM_image[num][ber][0] = max([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
                #MM_image[num][ber][1] = min([w_image[num][ber][0],w_image[num][ber][1],w_image[num][ber][2]])
            #w_images.append(MM_image)
            #w_images.append(w_image)
            
        a = img[0::2,0::2]
        b = img[0::2,1::2]
        c = img[1::2,0::2]
        d = img[1::2,1::2]
        
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(a)
        
        test = np.zeros((8,8))
        
        #print(img.numpy())
        
        LL = (a + b + c + d) / 4
        LH = (a - b + c - d) / 4
        HL = (a + b - c - d) / 4
        HH = (a - b - c + d) / 4
        
#          wave[:][:][0].assign(LL)
#          wave[:][:][1] = LH
#          wave[:][:][2] = HL
#          wave[:][:][3] = HH
#         wave(:,:,0) = LL
#         wave(:,:,1) = LH
#         wave(:,:,2) = HL
#         wave(:,:,3) = HH
        
        #print(wave)
        #images = np.array(images)

        #if ret_wavelet:
        #return np.array(w_image)
        return wave
        #else:
            #return np.array(images), np.array(labels)
        


In [74]:
if __name__ == '__main__':
   
    dcgan = DCGAN()
    
    r, c = 5, 5
    check_noise = np.random.uniform(-1, 1, (r * c, 100))
    dcgan.train(iterations=200000, batch_size=32, save_interval=1000,
                model_interval=5000, check_noise=check_noise, r=r,c=c)


Model: "sequential_73"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_90 (Conv2D)           (None, 8, 8, 48)          240       
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 8, 8, 128)         24704     
_________________________________________________________________
dropout_44 (Dropout)         (None, 8, 8, 128)         0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 128)               1048704   
_________________________________________________________________
dropout_45 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 1)               

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'input_28' with dtype float and shape [?,100]
	 [[node input_28 (defined at <ipython-input-73-016a4d780a08>:20) ]]
	 [[map_8/TensorArrayV2Stack/TensorListStack/_9]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'input_28' with dtype float and shape [?,100]
	 [[node input_28 (defined at <ipython-input-73-016a4d780a08>:20) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'input_28':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/lo

In [ ]:
import tensorflow as tf

if __name__ == '__main__':
    
    a = np.array([[1, 2]])
    b = np.array([1, 2])
    
    
    print(a.shape, b.shape)
    

    const1 = tf.constant(2)
    const2 = tf.constant(3, dtype=tf.float32)
    add_op = tf.add(const1, const2) # 2 + 3
    w = tf.zeros([3, 2])
    
#     with tf.Session() as sess:
#         result = sess.run(tf.add(w, const2))
#         print(result)
        
        

In [77]:
def func(x):
    a = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            a[i,j] = 10*i+j
    return a

def func2(x, adder):
    a = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            a[i,j] = 10*i+j + adder

def func3(x, adder):
    a = np.zeros(x.shape)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            a[i,j] = 10*i+j + adder
    return a

In [6]:
# params x: (?, ?, 3)
def wavelet_test(x, img, w, h):
    x = np.zeros(x.shape)
    
    h, w, _ = img.shape
    print(h, w)
#     for num in range(0, w//2):
#         for ber in range(0, h//2):
# #             print((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]))
#             x[num][ber][0].assign((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#             x[num][ber][1].assign((img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
#             x[num][ber][2].assign((img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4)
    return x

In [7]:
# params x: (?, ?, 3)
def wavelet_internal(x, img, w, h):
    x = np.zeros(x.shape)
    
    for num in range(0, w//2):
        for ber in range(0, h//2):
#             print((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]))
            x[num][ber][0].assign((img[num*2][ber*2] - img[num*2+1][ber*2] + img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
            x[num][ber][1].assign((img[num*2][ber*2] + img[num*2+1][ber*2] - img[num*2][ber*2+1] - img[num*2+1][ber*2+1]) / 4)
            x[num][ber][2].assign((img[num*2][ber*2] - img[num*2+1][ber*2] - img[num*2][ber*2+1] + img[num*2+1][ber*2+1]) / 4)
    return x

In [5]:
import copy
x = tf.Variable(lambda: tf.zeros((4,3)))
print(x)
y = copy.deepcopy(x)
tf.compat.v1.assign(x, x+2)
print(x)

# with tf.compat.v1.Session() as sess:
#     sess.run(tf.compat.v1.global_variables_initializer())
#     ret = sess.run(op)
#     print(type(ret)) # <class 'numpy.ndarray'>
#     print(ret)

<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.]], dtype=float32)>


TypeError: Fetch argument <tf.Variable 'UnreadVariable' shape=(4, 3) dtype=float32> has invalid type <class 'tensorflow.python.ops.resource_variable_ops._UnreadVariable'>, must be a string or Tensor. (Can not convert a _UnreadVariable into a Tensor or Operation.)

In [65]:
c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
e = tf.matmul(c, d)
print(e)
print(e.np())

Tensor("MatMul_2:0", shape=(2, 2), dtype=float32)


AttributeError: 'Tensor' object has no attribute 'np'

In [68]:
add = lambda a, b=1 : a + b

In [4]:
tf.executing_eagerly()

True

In [3]:
tf.compat.v1.enable_eager_execution()

<class 'type'>


In [6]:
!pwd

/tmp/py/in_report
